# Código para aplicação das LB direto no dado para rotulação

## Preparando o test da base ouro para aplicar as labels functions

In [49]:
import pandas as pd
import nltk
import scipy.stats
import sklearn
import sklearn_crfsuite
from sklearn_crfsuite import metrics

In [50]:
#apenas para testar o iob com spacy
#cria um novo dataset com as 2000 primeiras linhas
#csv_reader = pd.read_csv("/home/lucelia_vieira/Experimentos/ner/CSVs/REL_EXTRATO_CONTRATO_COMPLETO.csv", nrows=200)

In [51]:
#df = ["EXTRATO DO CONTRATO No 11/2019 PROCESSO: 00390-00003006/2019-92. ESPECIE: Contrato de Aquisicao de Bens pelo Distrito Federal no 11/2019. PARTES: Secretaria de Estado de Desenvolvimento Urbano e Habitacao do Distrito Federal - SEDUH/DF e INDUSTRIA DE AGUA MINERAL IBIA LTDA. DO OBJETO: Aquisicao de agua potavel de mesa sem gas e garrafao retornavel, destinados ao atendimento desta pasta, conforme demais especiGcacoes e condicoes estabelecidas no Termo de Referencia Anexo I do Edital de Pregao Eletronico no 0058/2018 - SCG/SEPLAG, da Proposta da empresa INDUSTRIA DE AGUA MINERAL IBIA LTDA de 11/06/2018 e da Lei no 8.666, de 21 de junho de 1993, que passam a integrar o presente Termo. DO VALOR: R$ 11.661,00 (onze mil, seiscentos e sessenta e um reais) conforme Nota de Empenho no 2019NE00180, emitida em 18/07/2019. Dotacao Orcamentaria P.T 15.122.6001.8517.0131 - Natureza da Despesa: 3390.30 - Fonte de Recursos: 100. DO PRAZO DE VIGENCIA: O Contrato tera vigencia de 06 (seis) meses a contar do dia 07/08/2019. DATA DE ASSINATURA: 22/07/2019. SIGNATARIOS: Pelo Distrito Federal: Mateus Leandro de Oliveira, na qualidade de Secretario de Estado. Pela Contratada: Eduardo Barros de Queiroz Rodrigues, na qualidade de Socio Administrador.", "EXTRATO DO CONTRATO Nº 06/2021 Processo: 00139-00001275/2021-00. Partes: DF - ADMINISTRAÇÃO REGIONAL DO CRUZEIRO X COMERCIAL DE ALIMENTOS PARK SUL. Objeto: O presente contrato tem por objeto a contratação de empresa especializada na área de alimentação para confecção e fornecimento de 50 KG DE BOLO CONFEITADO TIPO ANIVERSÁRIO COM MASSA PÃO DE LÓ COM RECHEIO DE BABA DE MOÇA, BEIJINHO OU DOCE DE LEITE E COBERTURA DE CHANTILLY DECORADO COM ACABAMENTO, no evento de comemoração ao aniversário de 62 anos do Cruzeiro no dia 30/11/2021. Valor do Contrato: R$ 2.000,00 (Dois Mil Reais). Dotação orçamentária: Unidade Orçamentária: 190113. Programa de Trabalho: 13.392.6219.3678.0136 . Natureza da Despesa: 339039. Fonte de Recursos: 120. Nota de Empenho: 2021NE00259. Valor do Empenho Inicial: R$ 2.000,00 (Dois Mil Reais), emitida em 26/11/2021 sob o evento 400091, na Modalidade 1   Ordinário. Fundamento Legal: Art. 24 da Lei nº 8.666/1993. Data da Assinatura: 29/11/2021. Signatários: Pelo DISTRITO FEDERAL: LUIZ EDUARDO GOMES DE PAULA PESSOA, na qualidade de Administrador Regional do Cruzeiro; pela COMERCIAL DE ALIMENTOS PARK SUL: GIVANILDES DE AGUIAR MORAIS, na qualidade de Gerente Geral."] 

In [52]:
#Carrega o dataset de teste para aplicacao das LB
#data = pd.read_csv('./CSVs/V2/Extrato_Contrato_Split_0_x_test_texto.csv', dtype=str)
#data = pd.read_csv('./CSVs/V2/Suspensao_Licitacao_Split_0_x_test_texto.csv', dtype=str)
#data = pd.read_csv('./CSVs/V2/Aditamento_Contrato_Split_0_x_test_texto.csv', dtype=str)
#data = pd.read_csv('./CSVs/V2/Anul_Revog_Licitacao_Split_0_x_test_texto.csv', dtype=str)
#data = pd.read_csv('./CSVs/V2/Aviso_Licitacao_Split_0_x_test_texto.csv', dtype=str)
data = pd.read_csv('./CSVs/V2/Extrato_Convenio_Split_1_x_test_texto.csv', dtype=str)


data

,Unnamed: 0,Texto
0,0,EXTRATO DO CONVENIO DE CONCESSAO DE ESTAGIO No...
1,1,EXTRATO DE CONVÊNIO Objeto : Acordo para celeb...
2,2,EXTRATO DE CONVENIO CONVENIO : 9064/2019 . ASS...
3,3,EXTRATO DE CONVENIO PROCESSO : 00054-00072737/...
4,4,EXTRATO DE CONVÊNIO CELEBRADO EM 27/10/2021 Co...
5,5,EXTRATO DO TERMO DE CONVÊNIO Processo : 00401-...
6,6,EXTRATO DE TERMO DE CONVÊNIO DE ESTÁGIO Proces...


In [53]:
#criando novo dataframe somente com os textos
df1 = data['Texto'].reset_index(drop=True)
#df1 = data['texto'].reset_index(drop=True)
df1

0    EXTRATO DO CONVENIO DE CONCESSAO DE ESTAGIO No...
1    EXTRATO DE CONVÊNIO Objeto : Acordo para celeb...
2    EXTRATO DE CONVENIO CONVENIO : 9064/2019 . ASS...
3    EXTRATO DE CONVENIO PROCESSO : 00054-00072737/...
4    EXTRATO DE CONVÊNIO CELEBRADO EM 27/10/2021 Co...
5    EXTRATO DO TERMO DE CONVÊNIO Processo : 00401-...
6    EXTRATO DE TERMO DE CONVÊNIO DE ESTÁGIO Proces...
Name: Texto, dtype: object

In [54]:
#len(df_sk)

In [55]:
# CODIGO DE APLICACAO DA SUPERVISAO FRACA
#Para rodar extrato de contrato descomente abaixo as duas linhas
#import dodfSkweak_v1_2
#skw = dodfSkweak_v1_2.SkweakContratos(df1)

#Para rodar aditamento de contrato descomente abaixo as duas linhas
#import dodfSkweak_Aditamento
#skw = dodfSkweak_Aditamento.SkweakAditamento(df1)

#Para rodar Suspensao descomente abaixo as duas linhas
#import dodfSkweak_Suspensao
#skw = dodfSkweak_Suspensao.SkweakSuspensao(df1)


#Para rodar Anulacao e Revogação
#import dodfSkweak_Anul_Revog
#skw = dodfSkweak_Anul_Revog.SkweakAnulRevog(df1)

#Para rodar Convenio
import dodfSkweak_Convenio
skw = dodfSkweak_Convenio.SkweakConvenio(df1)

skw.apply_label_functions()
skw.train_HMM_Dodf()

/Users/lucelia/Documents/GitHub/experiments/members/lucelia/venv/lib/python3.9/site-packages/spacy/language.py:1895: UserWarning: [W123] Argument disable with value ['ner', 'lemmatizer'] is used instead of ['senter'] as specified in the config. Be aware that this might affect other components in your pipeline.
  warnings.warn(


Starting iteration 1
Finished E-step with 7 documents
Starting iteration 2
Finished E-step with 7 documents
Starting iteration 3
Finished E-step with 7 documents
Starting iteration 4


         1        -278.7359             +nan
         2        -267.7598         +10.9761
         3        -265.0752          +2.6846


Finished E-step with 7 documents


         4        -262.6876          +2.3877


Write to ./data/convenio.spacy...done


In [56]:
skw.list_spans_specific(0)

[00080.00072007/ 2018-72]


In [57]:
#skw.list_spans_all()

In [58]:
data_label=skw.get_hmm_dataframe()
data_label

,numero_convenio,processo_gdf,valor_convenio,unidade_orcamentaria,programa_trabalho,natureza_despesa,nota_empenho,data_assinatura_convenio,text,labels,valor_contrato
0,,00080.00072007/ 2018-72,NaN,,,,,,EXTRATO DO CONVENIO DE CONCESSAO DE ESTAGIO No...,O O O O O O O O O O O B-processo_gdf I-process...,
1,,,NaN,,,,,Processo 041.001.171/2021,EXTRATO DE CONVÊNIO Objeto : Acordo para celeb...,O O O O O O O O O O O O O O O O O O O O O O O ...,
2,,,"1.817.180,00",,,17.122.6001.8517/ 6977-33 .90.39,,,EXTRATO DE CONVENIO CONVENIO : 9064/2019 . ASS...,O O O O O O O O O O O O O O O O O O O O O O O ...,
3,,00054-00072737/ 2018-72,NaN,,,,,09 de julho de 2019,EXTRATO DE CONVENIO PROCESSO : 00054-00072737/...,O O O O O B-processo_gdf I-processo_gdf O O O ...,
4,,,"0,00",,,,,,EXTRATO DE CONVÊNIO CELEBRADO EM 27/10/2021 Co...,O O O O O O O O O O O O O O O O O O O O O O O ...,
5,,00401-00012085/ 2021-06,NaN,,,,,,EXTRATO DO TERMO DE CONVÊNIO Processo : 00401-...,O O O O O O O B-processo_gdf I-processo_gdf O ...,
6,,00400-00022420/ 2021-02,NaN,,,,,,EXTRATO DE TERMO DE CONVÊNIO DE ESTÁGIO Proces...,O O O O O O O O O B-processo_gdf I-processo_gd...,


In [59]:
# converting to CSV file
#data_label.to_csv("./CSVs/ANUL_REVOG_SUPERVISÃO_FRACA_TESTE.csv")

In [60]:
corpus = skw.get_IOB()

In [61]:
len(corpus)

7

In [62]:
termos_skweak = []
debug=set()
labels_skweak = []
for ato in corpus:
    x = []
    y = []
    for termo in ato:
        termo_label = termo.split()
        x.append(termo_label[0])
        y.append(termo_label[1])
        debug.add(termo_label[1])
    termos_skweak.append(x)
    labels_skweak.append(y)


In [63]:
debug

{'B-data_assinatura_convenio',
 'B-natureza_despesa',
 'B-processo_gdf',
 'B-valor_convenio',
 'I-data_assinatura_convenio',
 'I-natureza_despesa',
 'I-processo_gdf',
 'O'}

In [64]:
len(termos_skweak)

7

In [65]:
termos_skweak_list = []
for i in termos_skweak:
   if not isinstance(i, list):
      termos_skweak_list .append(i)
   else:
      for j in i:
       termos_skweak_list .append(j)

len(termos_skweak_list )

1417

In [66]:
#termos_skweak_list

In [67]:
#apenas para conferencia
#df = pd.DataFrame(termos_skweak_list, columns=['Texto']) 
#df.to_csv('/home/lucelia_vieira/Experimentos/ner/CSVs/termos_skweak_list.csv')

In [68]:
labels_skweak_list = []
for i in labels_skweak:
   if not isinstance(i, list):
      labels_skweak_list.append(i)
   else:
      for j in i:
        labels_skweak_list.append(j)

len(labels_skweak_list)

1417

## Preparando o test da base ouro para fazer a validação

In [69]:
#Carrega o dataset de teste da base ouro para aplicar a validação
#df_gold_ato= pd.read_csv('./CSVs/Extrato_Contrato_Split_1_x_test.csv', dtype=str)
#df_gold_ato

In [70]:
#Carrega os atos do dataset de teste da base ouro
#https://pt.stackoverflow.com/questions/232208/ler-uma-lista-de-listas-em-txt-python-3
import pickle
#Contrato
#with open("./CSVs/V2/Extrato_Contrato_Split_0_x_test.txt", "rb") as file:
#Aditamento
#with open("./CSVs/V2/Aditamento_Contrato_Split_0_x_test.txt", "rb") as file:
#Suspensao
#with open("./CSVs/V2/Suspensao_Licitacao_Split_0_x_test.txt", "rb") as file:
#Anul e Revog
#with open("./CSVs/V2/Anul_Revog_Licitacao_Split_0_x_test.txt", "rb") as file: 
#Aviso Licitacao
#with open("./CSVs/V2/Aviso_Licitacao_Split_0_x_test.txt", "rb") as file: 
#Convenio
with open("./CSVs/V2/Extrato_Convenio_Split_0_x_test.txt", "rb") as file:

      
    x_test = pickle.load(file)

In [71]:
# Retira uma nível da lista dos atos do dataset de teste da base ouro
x_test_list = []
for i in x_test:
   if not isinstance(i, list):
      x_test_list.append(i)
   else:
      for j in i:
        x_test_list.append(j)

len(x_test_list)

1888

In [72]:
#Carrega os labels do dataset de teste da base ouro (Lista dentro de lista)
import pickle
#Contrato
#with open("./CSVs/V2/Extrato_Contrato_Split_0_y_test.txt", "rb") as file:
#Aditamento
#with open("./CSVs/V2/Aditamento_Contrato_Split_0_y_test.txt", "rb") as file:
#Suspensao
#with open("./CSVs/V2/Suspensao_Licitacao_Split_0_y_test.txt", "rb") as file:
#Anul e Revog
#with open("./CSVs/V2/Anul_Revog_Licitacao_Split_0_y_test.txt", "rb") as file:
#Aviso Licitacao
#with open("./CSVs/V2/Aviso_Licitacao_Split_0_y_test.txt", "rb") as file: 
#Convenio
with open("./CSVs/V2/Extrato_Convenio_Split_0_y_test.txt", "rb") as file:        
    y_test = pickle.load(file)

In [73]:
#  Retira uma nível da lista dos labels do dataset de teste da base ouro

y_test_list = []
for i in y_test:
   if not isinstance(i, list):
      y_test_list.append(i)
   else:
      for j in i:
       y_test_list.append(j)

len(y_test_list)

1888

In [74]:
len(termos_skweak_list)

1417

In [75]:
#Apenas checa se os tokens estão batento
resultados = []
for sk_ato , gold_ato in zip(termos_skweak_list, x_test_list):
    tupla=(sk_ato , gold_ato) 
    resultados.append(tupla)
    if sk_ato != gold_ato:
        print(sk_ato, gold_ato)

#for item in resultados:
#    print(item)


DO DE
DE ESPECIE
CONCESSAO :
DE CONVENIO
ESTAGIO No
No 06/
04/2019 2019-SES
Processo /
: DF
00080.00072007/ .
2018-72 PARTICIPES
- :
Partes a
: Secretaria
SEEDF de
X Estado
UNIGRAN de
EDUCACIONAL Saude
, (
mantenedora SES-
do DF
CENTRO )
UNIVERSITARIO ,
DA a
GRANDE Fundacao
DOURADOS de
- Ensino
UNIGRAN e
. Pesquisa
Objeto em
: Ciencias
a da
) Saude
a e
realizacao a
de ACAO
Estagio EDUCACIONAL
Curricular CLARETIANA
Obrigatorio ,
Supervisionado mantenedora
, do
sem CENTRO
remuneracao UNIVERSITARIO
, CLARETIANO
nas .
unidades OBJETO
escolares :
da concessao
Rede de
Publica area
de para
Ensino realizacao
do de
Distrito estagio
Federal curricular
ou e
nas Atividades
Unidades Praticas
Administrativas Supervisionadas
da nas
SEEDF estruturas
, organicas
para da
academicos SES-DF
regularmente e
matriculados entidades
no vinculadas
CENTRO ,
UNIVERSITARIO por
DA alunos
GRANDE regularmente
DOURADOS matriculados
- e
UNIGRAN que
, estejam
nos frequentando
cursos ,
de efetivamente
graduacao ,
/ o
lic

In [76]:
''' #labels_skweak = labels
numero_contrato = 0
numero_contrato_total = 0
B_numero_contrato = 0
numero_contrato_gold=[]
obj_sk=[]


data_assinatura_contrato = 0
data_assinatura_contrato_total = 0
B_data_assinatura_contrato = 0
data_assinatura_contrato_gold = []

unidade_orcamentaria = 0
unidade_orcamentaria_total = 0
B_unidade_orcamentaria = 0
unidade_orcamentaria_gold = []
teste=[]

programa_trabalho = 0
programa_trabalho_total = 0
B_programa_trabalho = 0
programa_trabalho_gold = []

for ato_sk , label_sk, ato_gold, label_gold in zip(termos_skweak_list, labels_skweak_list,  gold_x_test_list, gold_y_test_list):    
    if label_gold == "B-numero_contrato":
           #print("Teste")
        numero_contrato_total+=1
        B_numero_contrato=label_gold
        if label_gold == label_sk:
               numero_contrato+=1
        else:
            numero_contrato_gold.append(ato_gold)

    elif label_gold == "B-data_assinatura_contrato":
                
        data_assinatura_contrato_total+=1
        B_data_assinatura_contrato=label_gold
        if label_gold == label_sk:
            data_assinatura_contrato+=1
        else:
            data_assinatura_contrato_gold.append(ato_gold)    
    elif label_gold == "B-unidade_orcamentaria":    
        unidade_orcamentaria_total+=1
        B_unidade_orcamentaria=label_gold
        if label_gold == label_sk:
            unidade_orcamentaria+=1        
        else:
            unidade_orcamentaria_gold.append(ato_gold) 
                 
    elif label_gold == "B-programa_trabalho":      
        programa_trabalho_total+=1
        B_programa_trabalho=label_gold
        if label_gold == label_sk:
            programa_trabalho+=1
        else:
            programa_trabalho_gold.append(ato_gold)                
print("Tipo da entidade             Total Entidade Gold                 Total Entidade Skweak")                
print("B-numero_contrato",              "   ",numero_contrato_total,         "   ",numero_contrato)#,"      ","       ",numero_contrato_gold)
print("B-data_assinatura_contrato"," ",data_assinatura_contrato_total,"   ",data_assinatura_contrato)#,"      ",data_assinatura_contrato_gold)
print("B-unidade_orcamentaria",      "  ",unidade_orcamentaria_total,    "   ",unidade_orcamentaria)#,"          ",unidade_orcamentaria_gold)
print( "B-programa_trabalho",       "  ",programa_trabalho_total,       "   ",programa_trabalho)#,"             ",programa_trabalho_gold)

#print(programa_trabalho_gold)
#print(programa_trabalho_gold) '''

' #labels_skweak = labels\nnumero_contrato = 0\nnumero_contrato_total = 0\nB_numero_contrato = 0\nnumero_contrato_gold=[]\nobj_sk=[]\n\n\ndata_assinatura_contrato = 0\ndata_assinatura_contrato_total = 0\nB_data_assinatura_contrato = 0\ndata_assinatura_contrato_gold = []\n\nunidade_orcamentaria = 0\nunidade_orcamentaria_total = 0\nB_unidade_orcamentaria = 0\nunidade_orcamentaria_gold = []\nteste=[]\n\nprograma_trabalho = 0\nprograma_trabalho_total = 0\nB_programa_trabalho = 0\nprograma_trabalho_gold = []\n\nfor ato_sk , label_sk, ato_gold, label_gold in zip(termos_skweak_list, labels_skweak_list,  gold_x_test_list, gold_y_test_list):    \n    if label_gold == "B-numero_contrato":\n           #print("Teste")\n        numero_contrato_total+=1\n        B_numero_contrato=label_gold\n        if label_gold == label_sk:\n               numero_contrato+=1\n        else:\n            numero_contrato_gold.append(ato_gold)\n\n    elif label_gold == "B-data_assinatura_contrato":\n                \n

In [77]:
''' #Compara os labels reais e os preditos para avalir a rotulação fraca

#labels_skweak = labels
numero_contrato = 0
numero_contrato_total = 0
B_numero_contrato = 0
numero_contrato_gold=[]

data_assinatura_contrato = 0
data_assinatura_contrato_total = 0
B_data_assinatura_contrato = 0
data_assinatura_contrato_gold = []

processo_gdf = 0
processo_gdf_total = 0
B_processo_gdf = 0
processo_gdf_gold = []

unidade_orcamentaria = 0
unidade_orcamentaria_total = 0
B_unidade_orcamentaria = 0
unidade_orcamentaria_gold = []
teste=[]

programa_trabalho = 0
programa_trabalho_total = 0
B_programa_trabalho = 0
programa_trabalho_gold = []

#for text_sk, label_skweak, text_gold,  label_gold in zip(termos_skweak, labels_skweak, atos, labels):
for ato_sk , label_sk, ato_gold, label_gold in zip(termos_skweak_list, labels_skweak_list,  x_test_list, y_test_list):  
    #for text_skw, termo_skweak, text_ato, termo_gold in zip(text_sk, label_skweak, text_gold,  label_gold ):
       
       #print("termo swkeak", termo_skweak) 
       if label_gold == "B-numero_contrato":
           #print("Teste")
           numero_contrato_total+=1
           B_numero_contrato=label_gold
           if label_gold == label_sk:
               numero_contrato+=1
           else:
               numero_contrato_gold.append(ato_gold)

       elif label_gold == "B-data_escrito":    
            data_assinatura_contrato_total+=1
            B_data_assinatura_contrato=label_gold
            if label_gold == label_sk:
                data_assinatura_contrato+=1
            else:
                data_assinatura_contrato_gold.append(ato_gold)
       
       elif label_gold == "B-processo_gdf":        
            processo_gdf_total+=1
            B_processo_gdf=label_gold
            if label_gold == label_sk:
                processo_gdf+=1
            else:
                processo_gdf_gold.append(ato_gold)         
                

       elif label_gold == "B-unidade_orcamentaria":     
            unidade_orcamentaria_total+=1
            B_unidade_orcamentaria=label_gold
            if label_gold == label_sk:
                unidade_orcamentaria+=1
                
            else:
                unidade_orcamentaria_gold.append(ato_gold) 
                 
       elif label_gold == "B-programa_trabalho":  
            programa_trabalho_total+=1
            B_programa_trabalho=label_gold
            if label_gold == label_sk:
                programa_trabalho+=1
            else:
                programa_trabalho_gold.append(ato_gold)                
print("Tipo da entidade             Total Entidade Gold                 Total Entidade Skweak")                
print("B-numero_contrato",              "   ",numero_contrato_total,         "   ",numero_contrato,"      ","       ",numero_contrato_gold)
print("B-data_assinatura_contrato"," ",data_assinatura_contrato_total,"   ",data_assinatura_contrato,"      ",data_assinatura_contrato_gold)
print("B-processo_gdf"," ", processo_gdf_total,"   ",processo_gdf,"      ",processo_gdf_gold)

#print("B-unidade_orcamentaria",      "  ",unidade_orcamentaria_total,    "   ",unidade_orcamentaria)#,"          ",unidade_orcamentaria_gold)
#print( "B-programa_trabalho",       "  ",programa_trabalho_total,       "   ",programa_trabalho)#,"             ",programa_trabalho_gold)

#print(programa_trabalho_gold) '''

' #Compara os labels reais e os preditos para avalir a rotulação fraca\n\n#labels_skweak = labels\nnumero_contrato = 0\nnumero_contrato_total = 0\nB_numero_contrato = 0\nnumero_contrato_gold=[]\n\ndata_assinatura_contrato = 0\ndata_assinatura_contrato_total = 0\nB_data_assinatura_contrato = 0\ndata_assinatura_contrato_gold = []\n\nprocesso_gdf = 0\nprocesso_gdf_total = 0\nB_processo_gdf = 0\nprocesso_gdf_gold = []\n\nunidade_orcamentaria = 0\nunidade_orcamentaria_total = 0\nB_unidade_orcamentaria = 0\nunidade_orcamentaria_gold = []\nteste=[]\n\nprograma_trabalho = 0\nprograma_trabalho_total = 0\nB_programa_trabalho = 0\nprograma_trabalho_gold = []\n\n#for text_sk, label_skweak, text_gold,  label_gold in zip(termos_skweak, labels_skweak, atos, labels):\nfor ato_sk , label_sk, ato_gold, label_gold in zip(termos_skweak_list, labels_skweak_list,  x_test_list, y_test_list):  \n    #for text_skw, termo_skweak, text_ato, termo_gold in zip(text_sk, label_skweak, text_gold,  label_gold ):\n    

In [78]:

''' classes = ['B-numero_contrato',
 'B-processo_gdf',
 'I-processo_gdf',
 'B-data_assinatura_contrato',
 'I-numero_contrato',
 'I-data_assinatura_contrato'] '''


" classes = ['B-numero_contrato',\n 'B-processo_gdf',\n 'I-processo_gdf',\n 'B-data_assinatura_contrato',\n 'I-numero_contrato',\n 'I-data_assinatura_contrato'] "

In [79]:
len(labels_skweak_list)

1417

In [80]:
#classes

In [81]:
#Validação com sklearn
#metrics.flat_f1_score(y_test, labels_skweak, average='weighted', labels=classes)

In [82]:
#Validação com seqeval
from seqeval.metrics import f1_score
f1_score(y_test, labels_skweak, average='weighted')

ValueError: Found input variables with inconsistent numbers of samples:
[249, 323, 153, 246, 328, 371, 218]
[200, 82, 352, 169, 118, 125, 371]

In [ ]:
# classification_report Seqeval
import numpy as np
from seqeval.metrics import classification_report
from seqeval.scheme import IOB2

report = classification_report(y_test, labels_skweak, output_dict=False, mode='strict', scheme=IOB2)
print(report)


                          precision    recall  f1-score   support

data_assinatura_convenio       0.00      0.00      0.00         3
     entidade_convenente       0.00      0.00      0.00         7
        nome_responsavel       0.00      0.00      0.00         2
         numero_convenio       1.00      0.40      0.57         5
         objeto_convenio       0.00      0.00      0.00         5
        orgao_concedente       0.00      0.00      0.00         5
            processo_gdf       1.00      0.60      0.75         5
       vigencia_convenio       0.00      0.00      0.00         3

               micro avg       1.00      0.14      0.25        35
               macro avg       0.25      0.12      0.17        35
            weighted avg       0.29      0.14      0.19        35



/Users/lucelia/Documents/GitHub/experiments/members/lucelia/venv/lib/python3.9/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# classification_report Seqeval
import numpy as np
from seqeval.metrics import classification_report
from seqeval.scheme import IOB2

report = classification_report(y_test, labels_skweak, output_dict=True, mode='strict', scheme=IOB2)
print(report)
np.save("./Results/METRICA_EXTRATO_CONTRATO_LB_DIRETO_0.npy", report)

{'cnpj_entidade_convenente': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 3}, 'cnpj_orgao_concedente': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 3}, 'data_assinatura_convenio': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 7}, 'entidade_convenente': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 8}, 'fonte_recurso': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 2}, 'natureza_despesa': {'precision': 1.0, 'recall': 0.75, 'f1-score': 0.8571428571428571, 'support': 4}, 'nota_empenho': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 4}, 'numero_convenio': {'precision': 0.75, 'recall': 0.375, 'f1-score': 0.5, 'support': 8}, 'objeto_contrato': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 1}, 'objeto_convenio': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 9}, 'orgao_concedente': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 5}, 'processo_gdf': {'pr

/Users/lucelia/Documents/GitHub/experiments/members/lucelia/venv/lib/python3.9/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
#classification_report sklearn
report = metrics.flat_classification_report(
    #y_test[:200], y_pred[:200], labels=classes, digits=3))
   gold_y_test, labels_skweak, labels=classes, digits=3, output_dict=False)
print(report)

NameError: name 'gold_y_test' is not defined

In [ ]:
import numpy as np
report = metrics.flat_classification_report(
    #y_test[:200], y_pred[:200], labels=classes, digits=3))
    gold_y_test, labels_skweak, labels=classes, digits=3, output_dict=True)
print(report)
np.save("./Results/METRICA_EXTRATO_CONTRATO_LB_DIRETO_V1_0.npy", report)